# ニューラルネットワークの誤答に関する調査

<a href="05.ipynb"><b>グリッドサーチにより最適化が行われたモデル</b></a>でも、数字だけを見れば1.5%〜2.5%の誤答等が出るようです。

こちらの誤答例やその原因について調査しました。

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

### (1-1) テストデータをコピー

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    'test_daikin_conversation.csv',
    'test_benefitone_conversation.csv',
    'test_septeni_conversation.csv',
    'test_ptna_conversation.csv',
]
temp_path = TestTool.copy_testdata_csv(learning_dir, csv_file_names)

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_septeni_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv]


## (2) モデルの準備

学習(GridSearchCV.fit)--->評価(Evaluator.evaluate)の流れで実行させます。

グリッドサーチは省略しております。

In [3]:
from time import time
import numpy as np

from sklearn.grid_search import GridSearchCV
from sklearn.neural_network import MLPClassifier

from learning.core.evaluator import Evaluator

def fit_and_cross_validation(path):
    '''
        訓練データのTF-IDFベクターを作成
    '''
    basename = os.path.basename(path)
    print("prepare_tf_idf_vectors: dataset=%s..." % basename)
    t0 = time()

    X, y, vectorizer = TestTool.prepare_tf_idf_vectors(path)
    print("prepare_tf_idf_vectors: done in %0.3fs." % (time() - t0))

    '''
        訓練データ全体を使用して学習実施
        レイヤーはデフォルトの1層
        レイヤーに100件ユニットを生成する設定
    '''
    print("MLPClassifier: fitting...")
    t0 = time()

    cls = MLPClassifier(hidden_layer_sizes=(100,), max_iter=10000,
                        activation='logistic', shuffle=False, random_state=0)
    estimator = cls.fit(X, y)
    print("MLPClassifier: done in %0.3fs." % (time() - t0))

    ''' 
        クロスバリデーション（モデル評価フェーズ）を実施
        プロダクションと同様、Evaluator クラスを使用して評価します
    '''
    print("Evaluator: evaluating...")
    t0 = time()

    evaluator = Evaluator()
    evaluator.evaluate(estimator, X, y, threshold=0.5)
    print("Evaluator: done in %0.3fs." % (time() - t0))
    
    return (basename, X, y, vectorizer, estimator, evaluator)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### (2-1) モデル作成／評価

In [4]:
list_of_classifiers = []
for path in temp_path:
    classifier = fit_and_cross_validation(path)
    list_of_classifiers.append(classifier)

2017/04/06 PM 04:23:23 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 17443
2017/04/06 PM 04:23:23 TextArray#__init__ start


prepare_tf_idf_vectors: dataset=test_daikin_conversation.csv...


2017/04/06 PM 04:23:33 TextArray#to_vec start
2017/04/06 PM 04:23:33 TextArray#to_vec end


prepare_tf_idf_vectors: done in 10.189s.
MLPClassifier: fitting...


2017/04/06 PM 04:26:19 self.threshold: 0.5


...

prepare_tf_idf_vectors: done in 1.834s.
MLPClassifier: fitting...


2017/04/06 PM 04:30:11 self.threshold: 0.5


MLPClassifier: done in 16.124s.
Evaluator: evaluating...


2017/04/06 PM 04:30:26 Evaluator#evaluate#elapsed time: 14404.474020 ms
2017/04/06 PM 04:30:26 accuracy: 0.978965819457


0.978965819457
Evaluator: done in 14.407s.


### (2-2) 結果の確認

In [5]:
from learning.core.persistance import Persistance
bot_id = 9000

for classifier in list_of_classifiers:
    basename, X, y, vectorizer, estimator, evaluator = classifier
    params = estimator.get_params()

    print('[%s] best parameter: hidden_layer_sizes=%s, accuracy=%0.6f' % (
        basename, str(params['hidden_layer_sizes']), evaluator.accuracy
    ))
    '''
        分析のため、シリアライズします
    '''
    bot_id += 1
    Persistance.dump_model(estimator, bot_id)
    Persistance.dump_vectorizer(vectorizer, bot_id)

[test_daikin_conversation.csv] best parameter: hidden_layer_sizes=(100,), accuracy=0.985557
[test_benefitone_conversation.csv] best parameter: hidden_layer_sizes=(100,), accuracy=0.985437
[test_septeni_conversation.csv] best parameter: hidden_layer_sizes=(100,), accuracy=0.974074
[test_ptna_conversation.csv] best parameter: hidden_layer_sizes=(100,), accuracy=0.978966


## (3) 調査

### (3-1) 誤答結果の抽出

正解率が比較的低かった[test_septeni_conversation.csv]をサンプルとして使用します。

In [6]:
basename, X, y, vectorizer, estimator, evaluator = list_of_classifiers[2]
params = estimator.get_params()

print('[%s] accuracy=%0.6f, output=%d' % (
    basename, evaluator.accuracy, estimator.n_outputs_
))

[test_septeni_conversation.csv] accuracy=0.974074, output=165


#### 訓練データを予測にかけます

この中で不正解かつ閾値が上回っているものを抽出します。

（注：クロスバリデーション[Shuffle&Split]時に不正解になったものを含みます）

In [7]:
from sklearn.cross_validation import ShuffleSplit
from sklearn.cross_validation import cross_val_score

error_indexes = [] # 誤答データが入っているインデックス
unable_indexes = [] # 回答不能データが入っているインデックス

def __accuracy_score(estimator, X, y):
    y_pred = estimator.predict(X)
    probabilities = estimator.predict_proba(X)
    max_probabilities = np.max(probabilities, axis=1)

    # 予測結果と実際を比較
    bools = y == y_pred
    # しきい値を超えているもののみをTrueにする
    bools = bools == (max_probabilities > 0.5)
    # 正当率を算出
    score = np.sum(bools) / np.size(bools, axis=0)

    '''
        誤答分／回答不能分のみ抽出
    '''
    for i, _ in enumerate(y):
        if y[i] != y_pred[i] and max_probabilities[i] > 0.5: # これは誤答
            error_indexes.append((i, X[i], y[i], y_pred[i], max_probabilities[i]))

        if y[i] == y_pred[i] and max_probabilities[i] <= 0.5: # これは回答不能
            unable_indexes.append((i, X[i], y[i], y_pred[i], max_probabilities[i]))

    '''
        検算
    '''
    print('__accuracy_score: score=%0.6f(%d/%d), error detected=%d(error=%d, unable=%d)' % (
        score, 
        np.sum(bools), 
        np.size(bools, axis=0), 
        np.size(bools, axis=0)-np.sum(bools), # <---ここまでが accuracy 算出に用いたデータ
        len(error_indexes), # <---誤答分
        len(unable_indexes) # <---回答不能分
    ))

    return score

'''
    クロスバリデーション[Shuffle&Split]時に不正解になったものを抽出するため、
    訓練データを再度分割
'''
cv = ShuffleSplit(X.shape[0], n_iter=1, test_size=0.25, random_state=0)
__accuracy = np.mean(cross_val_score(estimator, X, y, cv=cv, scoring=__accuracy_score))

__accuracy_score: score=0.974074(526/540), error detected=14(error=12, unable=2)


### (3-2) 誤答されたデータのfeatureを確認

In [8]:
for i, __X, label, pred, max_proba in error_indexes:
    arr = __X.toarray()[0]
    dump_str = TestTool.dump_features(arr, vectorizer.vocabulary_)
    print('index=%d%s, label=%d --> predicted as %d(proba=%0.3f)' % (
        i, dump_str, label, pred, max_proba))

index=36[モニタ=0.707 不調=0.707], label=4613 --> predicted as 4577(proba=0.646)
index=47[sap=0.577 パスワード=0.577 忘れる=0.577], label=4441 --> predicted as 4503(proba=0.624)
index=60[sap=0.500 ない=0.500 ログイン=0.500 出来る=0.500], label=4524 --> predicted as 4515(proba=0.738)
index=130[パスワード=0.707 変更=0.707], label=4507 --> predicted as 4531(proba=0.747)
index=152[アカウント=0.577 競合=0.577 解消=0.577], label=4579 --> predicted as 4627(proba=0.931)
index=285[pw=0.577 忘れる=0.577 販管=0.577], label=4441 --> predicted as 4503(proba=0.733)
index=352[アカウント=0.577 競合=0.577 解消=0.577], label=4579 --> predicted as 4627(proba=0.931)
index=396[ない=0.500 ログイン=0.500 出来る=0.500 販管=0.500], label=4524 --> predicted as 4515(proba=0.617)
index=432[モニタ=0.707 不調=0.707], label=4613 --> predicted as 4577(proba=0.646)
index=461[windows=0.500 ない=0.500 ログイン=0.500 出来る=0.500], label=4583 --> predicted as 4420(proba=0.673)
index=503[pw=0.577 忘れる=0.577 販管=0.577], label=4441 --> predicted as 4503(proba=0.733)
index=505[pw=0.500 ログイン=0.500 忘れる=0

#### 4613 ---> 4577（2件）、4441 ---> 4503（3件）、4579 ---> 4627（2件）と誤答されるケースについて見てみます。

サンプル数の偏りがあったというわけではなさそうです。

In [9]:
[item for item in TestTool.count_sample_by_label(y) if item[0] in [4613,4577,4441,4503,4579,4627]]

[[4441, 22], [4503, 22], [4577, 8], [4579, 10], [4613, 8], [4627, 8]]

#### ボキャブラリIDを取得しておきます

In [10]:
[(k,v) for (k,v) in vectorizer.vocabulary_.items() if k in ['モニタ','不調']]

[('モニタ', 213), ('不調', 237)]

#### 出力レイヤーにおけるラベルのインデックスを取得しておきます

In [11]:
np.where(estimator.classes_==4613)

(array([148]),)

In [12]:
np.where(estimator.classes_==4577)

(array([112]),)

#### MLP内における feature の重み付けを確認

MLPモデル内の全ユニット（100件）に対する影響変数をprintしてみました。

「モニタ」が複数のユニットに対して強い特徴を示しているようです（2を超える値が頻出）

複数クラスに対して、影響が強いfeatureであると感じられます。

他方「不調」はあまり重み付けがされないことが伺えます（1.5を超える値がない）

In [13]:
'''
     feature「モニタ」に対する、ユニットごとの影響変数。
     値が 1.0 を超えている要素が多い
     ---> このfeatureが多くのユニットに影響を与えている可能性あり
'''
monitor = estimator.coefs_[0][213] # モニタ
monitor

array([ 1.75124299,  1.71858051, -1.71767921, -1.99715393,  1.80668973,
        1.98612823, -0.81741437, -1.51923621, -1.43344832,  0.97137116,
       -1.08127899, -1.31551312,  1.41857381, -1.46749987, -1.5003903 ,
        1.9655057 , -1.49038115, -0.09915292, -2.18583265,  2.1207377 ,
       -1.76189853, -1.12438231, -2.00025381,  2.22041372,  2.13069815,
       -1.29328061,  0.32684533,  1.55841729, -1.41472041,  2.0277358 ,
       -1.51712519, -1.61757326,  1.38591144, -1.04943509,  0.60662344,
       -2.02939331, -1.50090554,  1.68563419, -1.64366229,  1.71088974,
        2.42956051, -1.71804918,  1.5642658 , -1.35216496, -1.92300335,
       -1.87322252, -1.81612381, -0.44403946, -1.61742105, -0.73738901,
       -1.55416245, -1.62094359,  1.8125792 ,  1.93053547,  1.46247901,
       -0.18711195, -1.5544135 ,  0.94745708, -1.40768527, -0.60831401,
       -1.64367182, -1.85771126,  0.50493039, -0.92488594,  1.72777314,
        2.11566165,  2.06440852,  1.68753153,  1.68104491, -2.03

In [14]:
'''
    feature「モニタ」から誘導されるであろう
    ユニットのインデックスを抽出してみます
'''
monitor_index = np.where(monitor > 1.4)
monitor_index

(array([ 0,  1,  4,  5, 12, 15, 19, 23, 24, 27, 29, 37, 39, 40, 42, 52, 53,
        54, 64, 65, 66, 67, 68, 72, 78, 80, 86, 88, 89, 90, 91]),)

In [15]:
'''
     feature「不調」に対する、ユニットごとの影響変数。
     値が 1.0 を超えている要素がすくない
     ---> このfeatureが正解ラベルに誘導される可能性が低そう
'''
fuchou = estimator.coefs_[0][237]
fuchou

array([ 0.8159311 ,  0.77335313, -0.55941609, -0.59614403,  0.42339188,
        0.80956631, -1.11634515, -0.42412606, -0.88006043,  0.4977679 ,
       -0.83870146, -1.01352894,  0.8572597 ,  0.08555895,  0.05060537,
        0.49097145, -0.5319267 ,  1.32069518,  0.19704951,  0.93646919,
       -0.41247626, -0.84295712, -0.66733325,  0.7103349 ,  0.43625168,
       -0.98725756, -1.10532095,  0.59508651, -0.54341948,  0.45619787,
       -0.69581328, -0.67628005, -1.13576168, -0.95190457,  1.10845446,
       -0.93263386,  0.03395297,  0.44358973, -0.95834706,  0.52342216,
        0.65474784, -0.77665401,  0.07442442,  0.35896115, -0.21881334,
       -0.05545878, -0.65747264,  1.41356214, -0.91584137,  1.44558223,
       -0.62054166, -0.49290181,  0.55834308,  0.71659792, -0.88938439,
       -1.07417931, -0.89983388, -0.61589732,  1.28465698,  0.96236334,
       -0.1165838 , -0.48709424,  0.99555546, -1.06414608,  0.71944496,
        0.34302023,  0.51056792,  0.65100236, -0.02299371, -0.46

In [16]:
'''
    feature「不調」から誘導されるであろう
    ユニットのインデックスを抽出してみます
'''
fuchou_index = np.where(fuchou > 1.4)
fuchou_index

(array([47, 49, 71, 83]),)

#### 参考：正解ラベルと、誤答されたラベルの重み付けを確認

こちらも、MLPモデル内の全ユニット（100件）に対する影響変数をprintしてみました。

推測ですが、feature「モニタ」の影響が 4613（正解ラベル）よりも強い 4577 に引っ張られてしまったのでは・・・と推測しています

In [17]:
'''
     正解ラベル 4613 に誘導される
     出力レイヤーのユニットごとの影響変数
'''
seikai = estimator.coefs_[1].T[148]
seikai

array([ 0.49083822,  0.39070405, -0.46630306, -0.56690567,  0.39587314,
        0.458893  , -0.69106909, -0.35285143, -0.8012782 ,  0.13501075,
       -0.5256996 , -0.83588898,  0.49312897, -0.20555853, -0.22076135,
        0.28573533, -0.53927282,  0.36693938, -0.22428648,  0.48732519,
       -0.49123165, -0.40390388, -0.53447728,  0.40712811,  0.34846273,
       -0.70967361, -0.06981092,  0.42967023, -0.41072364,  0.38455513,
       -0.52132082, -0.4901413 , -0.0427044 , -0.76118532,  0.33082418,
       -0.54392046, -0.12918591,  0.35526643, -0.74846999,  0.39378736,
        0.32159746, -0.75933578,  0.21687621, -0.15349672, -0.24769814,
       -0.28844535, -0.61767084,  0.30320539, -0.56628316,  0.31922049,
       -0.49662856, -0.49902541,  0.35734677,  0.5101611 ,  0.14307261,
       -0.53899004, -0.49574514,  0.12743199,  0.13267789,  0.27969767,
       -0.28825158, -0.26553955,  0.30257418, -0.57702439,  0.47291031,
        0.29105785,  0.40841158,  0.42935141,  0.18675321, -0.38

In [18]:
'''
    feature「モニタ」から誘導されたであろう
    ユニットの影響変数を抽出してみます
'''
seikai_coef = seikai[monitor_index]
seikai_coef[seikai_coef > 0.5] # 0.5を超えているものだけ抽出

array([ 0.5101611 ,  0.50424246])

In [19]:
'''
    feature「不調」から誘導されたであろう
    ユニットの影響変数を抽出してみます
'''
seikai_coef = seikai[fuchou_index]
seikai_coef[seikai_coef > 0.5] # 0.5を超えているものだけ抽出

array([], dtype=float64)

In [20]:
'''
     誤答先ラベル 4577 に誘導される
     出力レイヤーのユニットごとの影響変数
'''
gotou = estimator.coefs_[1].T[112] # 誤答先のラベル
gotou

array([ 0.14199417,  0.39612735, -0.63403812, -0.62520682,  0.32502789,
        0.26370443,  0.39026281, -0.38884786, -0.18357886,  0.12606036,
       -0.14074037,  0.06317695, -0.02259517, -0.81314361, -0.56339883,
        0.33551269, -0.52536694, -0.71913637, -0.32333056, -0.04576833,
       -0.87880649,  0.03702268, -0.3901105 ,  0.26541143,  0.46416539,
        0.08139097,  0.53837967,  0.50603368, -0.48883159,  0.65708343,
       -0.47677993, -0.4245336 ,  0.27913033, -0.01411367, -0.4289071 ,
       -0.17287753, -0.39846825,  0.59034191, -0.17846167,  0.48458828,
        0.20601362, -0.62614346,  0.54961127, -0.94294814, -0.45719657,
       -0.69836234, -0.6081932 , -0.80170637,  0.03577268, -0.57171625,
       -0.41563182, -0.66670066,  0.42155077,  0.43222125,  0.44125076,
        0.29385293,  0.00881336,  0.42668363, -0.46498831, -0.13613934,
       -0.64496049, -0.21968882, -0.21648138, -0.07291517,  0.31166095,
        0.49803022,  0.5888137 ,  0.38983786,  0.59818746, -0.41

In [21]:
'''
    feature「モニタ」から誘導されたであろう
    ユニットの影響変数を抽出してみます
'''
gotou_coef = gotou[monitor_index]
gotou_coef[gotou_coef > 0.5] # 0.5を超えているものだけ抽出

array([ 0.50603368,  0.65708343,  0.59034191,  0.54961127,  0.5888137 ,
        0.59818746,  0.5129703 ])

In [22]:
'''
    feature「不調」から誘導されたであろう
    ユニットの影響変数を抽出してみます
'''
gotou_coef = gotou[fuchou_index]
gotou_coef[gotou_coef > 0.5] # 0.5を超えているものだけ抽出

array([], dtype=float64)

### (3-3) 回答不能とされたデータのfeatureを確認

In [23]:
for i, __X, label, pred, max_proba in unable_indexes:
    arr = __X.toarray()[0]
    dump_str = TestTool.dump_features(arr, vectorizer.vocabulary_)
    print('index=%d%s, label=%d --> predicted as %d(proba=%0.3f)' % (
        i, dump_str, label, pred, max_proba))

index=85[ない=0.577 無線=0.577 繋がる=0.577], label=4495 --> predicted as 4495(proba=0.442)
index=446[vpn=0.500 ない=0.500 出来る=0.500 接続=0.500], label=4473 --> predicted as 4473(proba=0.398)


#### こちらは単純に、サンプルが少なかったためと推測されます。

（4495=4件、4473=2件）

In [24]:
[item for item in TestTool.count_sample_by_label(y) if item[0] in [4495,4473]]

[[4473, 2], [4495, 4]]